# Preamble

In [ ]:
%load_ext autoreload
%autoreload 0

In [ ]:
%autoreload

## Imports

In [ ]:
idxwhere??

In [ ]:
import xarray as xr
import sqlite3
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from lib.pandas_util import idxwhere
import warnings
import torch
import pyro

import sfacts as sf
from sfacts.model_zoo import (
    full_metagenotype,
    full_metagenotype_no_missing,
    full_metagenotype_pp_pi,
    full_metagenotype_dirichlet_rho,
    full_metagenotype_dirichlet_rho_no_missing,
    full_metagenotype_pp_pi_no_missing,
#     full_metagenotype_special_meta,
#     full_metagenotype_no_missing_special_meta,
)


from lib.project_style import color_palette, major_allele_frequency_bins
from lib.project_data import metagenotype_db_to_xarray

In [ ]:
warnings.filterwarnings(
    "ignore",
    message="torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.",
    category=torch.jit.TracerWarning,
#     module="trace_elbo",  # FIXME: What is the correct regex for module?
#     lineno=5,
)

## Style

## Load Data

In [ ]:
con = sqlite3.connect('data/core.a.proc.gtpro.2.denorm.db')

In [ ]:
species_of_interest = '104345'

In [ ]:
_mgen_all = pd.read_sql(f"""
SELECT
  lib_id
, species_position
, reference_tally
, alternative_tally
FROM snp_x_lib
JOIN lib USING (lib_id)
WHERE species_id = '{species_of_interest}'
  AND lib_type = 'metagenome'
""", con=con, index_col=['lib_id', 'species_position'])

mgen_all = sf.data.Metagenotypes(metagenotype_db_to_xarray(_mgen_all))

In [ ]:
mgen_filt = (
    mgen_all
    .select_variable_positions(thresh=0.05)
    .select_samples_with_coverage(0.05)
)
mgen_ss = mgen_filt.random_sample(position=1000)

In [ ]:
mgen_filt.data

In [ ]:
sf.plot.plot_depth(mgen_ss.to_world())

In [ ]:
sf.plot.plot_metagenotype(
    mgen_ss.to_world(),
)

In [ ]:
nstrain = 100
nposition = min(int(5e3), mgen_filt.sizes['position'])

est, (genotypes, missingness) = (
# est, _ = (
    sf.workflow.fit_subsampled_metagenotype_collapse_strains_then_iteratively_refit_full_genotypes(
        full_metagenotype_no_missing.model_structure,
#         mgen_ss,
        mgen_filt,  # .random_sample(position=nposition),
        nstrain=nstrain,
        hyperparameters=dict(
            gamma_hyper=0.2,
#             delta_hyper_r=0.8,
#             delta_hyper_temp=0.01,
            rho_hyper=0.5,
            pi_hyper=0.5,
            epsilon_hyper_mode=0.01,
            epsilon_hyper_spread=1.5,
            alpha_hyper_hyper_mean=1000.0,
            alpha_hyper_hyper_scale=0.5,
            alpha_hyper_scale=2.0,
#             m_hyper_hyper_r_mean=10.,
#             m_hyper_hyper_r_scale=1.,
#             m_hyper_r_scale=1.,
        ),
        nposition=nposition,
        diss_thresh=0.02,
        frac_thresh=1e-3,
        stage2_hyperparameters=dict(gamma_hyper=1.0),
        device='cpu',
        dtype=torch.float64,
        quiet=False,
        estimation_kwargs=dict(
            lagA=10,
            lagB=100,
            opt=pyro.optim.Adamax({"lr": 1.0}, {"clip_norm": 100}),
            seed=2,
            jit=True,
        )
    )
)

In [ ]:
sf.plot.plot_metagenotype(
    est,
    col_linkage_func=lambda w: w.communities.linkage(dim="sample"),
    col_colors_func=lambda w: xr.Dataset(dict(
        focal=w.sample.isin(['SS01009.m', 'SS01057.m']),
        depth=w.metagenotypes.mean_depth().pipe(np.log),
        mgen_entropy=w.metagenotypes.entropy('sample'),
        comm_entropy=w.communities.entropy('sample'),
    )),
)

In [ ]:
sf.plot.plot_genotype(
    est,
    col_linkage_func=lambda w: w.metagenotypes.linkage(dim="position"),
    row_linkage_func=lambda w: w.genotypes.linkage(dim="strain"),
    row_colors_func=lambda w: xr.Dataset(dict(
        depth=(w.communities.data * w.metagenotypes.mean_depth()).mean("sample").pipe(np.log),
        entropy=w.genotypes.entropy(),
    )),
)

In [ ]:
sf.plot.plot_community(
    est,
#     col_linkage_func=lambda w: w.metagenotypes.linkage(dim="sample"),
    row_linkage_func=lambda w: w.genotypes.linkage(dim="strain"),
#     norm=None,
    col_colors_func=lambda w: xr.Dataset(dict(
        focal=w.sample.isin(['SS01009.m', 'SS01057.m']),
        depth=w.metagenotypes.mean_depth().pipe(np.log),
        mgen_entropy=w.metagenotypes.entropy('sample'),
        comm_entropy=w.communities.entropy('sample'),
    )),
    norm=mpl.colors.PowerNorm(1/5),
)

In [ ]:
sf.plot.plot_missing(est)

In [ ]:
est.communities.sel(sample=['SS01009.m', 'SS01057.m']).to_series().sort_values(ascending=False).head(5)

In [ ]:
sf.plot.plot_genotype(
    est.genotypes.mlift('sel', strain=[22, 24, 35]).to_world()
)

In [ ]:
_drplt_all = pd.read_sql(f"""
SELECT
  lib_id
, species_position
, reference_tally
, alternative_tally
FROM snp_x_lib
JOIN lib USING (lib_id)
WHERE species_id = '{species_of_interest}'
  AND lib_type = 'droplet'
""", con=con, index_col=['lib_id', 'species_position'])

drplt_all = sf.data.Metagenotypes(metagenotype_db_to_xarray(_drplt_all))

In [ ]:
drplt_filt = drplt_all.mlift('reindex', position=mgen_filt.position, fill_value=0).select_samples_with_coverage(0.05)

In [ ]:
d = sf.data.Genotypes.concat(dict(
    est=est.genotypes.mlift('sel', strain=[22, 24, 35]),
    other=est.genotypes,
    drplt=drplt_filt.to_estimated_genotypes(),
    mgen=mgen_filt.mlift('sel', sample=['SS01009.m', 'SS01057.m']).to_estimated_genotypes(),
), dim='strain')

sf.plot.plot_genotype(
    d.to_world(),
    row_colors_func=None,
    col_colors_func=None,
)

In [ ]:
nstrain = 100
nposition = min(int(5e3), mgen_filt.sizes['position'])

est2 = (
    sf.workflow.fit_subsampled_metagenotype_collapse_strains_then_iteratively_refit_full_genotypes(
        full_metagenotype_no_missing.model_structure,
#         mgen_ss,
        mgen_filt.random_sample(position=nposition),
        nstrain=nstrain,
        hyperparameters=dict(
            gamma_hyper=0.2,
#             delta_hyper_r=0.8,
#             delta_hyper_temp=0.01,
            rho_hyper=0.5,
            pi_hyper=1.0,
            epsilon_hyper_mode=0.01,
            epsilon_hyper_spread=1.5,
            alpha_hyper_hyper_mean=1000.0,
            alpha_hyper_hyper_scale=0.5,
            alpha_hyper_scale=2.0,
#             m_hyper_hyper_r_mean=10.,
#             m_hyper_hyper_r_scale=1.,
#             m_hyper_r_scale=1.,
        ),
        nposition=nposition,
        diss_thresh=0.02,
        frac_thresh=1e-3,
        stage2_hyperparameters=dict(gamma_hyper=1.0),
        device='cpu',
        dtype=torch.float64,
        quiet=False,
        estimation_kwargs=dict(
            lagA=10,
            lagB=100,
            opt=pyro.optim.Adamax({"lr": 1.0}, {"clip_norm": 100}),
            seed=2,
            jit=True,
        )
    )
)

In [ ]:
sf.plot.plot_community(
    est2,
#     col_linkage_func=lambda w: w.metagenotypes.linkage(dim="sample"),
    row_linkage_func=lambda w: w.genotypes.linkage(dim="strain"),
#     norm=None,
    col_colors_func=lambda w: xr.Dataset(dict(
        focal=w.sample.isin(['SS01009.m', 'SS01057.m']),
        depth=w.metagenotypes.mean_depth().pipe(np.log),
        mgen_entropy=w.metagenotypes.entropy('sample'),
        comm_entropy=w.communities.entropy('sample'),
    )),
    norm=mpl.colors.PowerNorm(1/2),
)

In [ ]:
e = est2

d = e.communities.sel(sample=['SS01009.m', 'SS01057.m']).to_series()
strain_list = d[d > 0.01].reset_index().strain.unique()
print(strain_list)
d.sort_values(ascending=False).head(10)

In [ ]:
e = est2

sf.plot.plot_genotype(
    e.genotypes.mlift('sel', strain=strain_list).to_world()
)

In [ ]:
e = est2

d = sf.data.Genotypes.concat(dict(
    est=e.genotypes.mlift('sel', strain=strain_list),
    other=e.genotypes,
    mgen=mgen_filt.metagenotypes.mlift('sel', sample=['SS01009.m', 'SS01057.m']).mlift('sel', position=e.position).to_estimated_genotypes(),
    drplt=drplt_filt.mlift('sel', position=e.position).to_estimated_genotypes(),
), dim='strain')

sf.plot.plot_genotype(
    d.to_world(),
    row_colors_func=None,
    col_colors_func=None,
#     row_linkage_func=None,
#     col_linkage_func=None,
#     col_cluster=False,
#     row_cluster=False,
)